In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensornetwork as tn
tn.set_default_backend("tensorflow")
class TNLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(TNLayer, self).__init__()
    # Create the variables for the layer.
    self.a_var = tf.Variable(tf.random.normal(shape=(32, 32, 2),
                                              stddev=1.0/32.0),
                             name="a", trainable=True)
    self.b_var = tf.Variable(tf.random.normal(shape=(32, 32, 2),
                                              stddev=1.0/32.0),
                             name="b", trainable=True)
    self.bias = tf.Variable(tf.zeros(shape=(32, 32)),
                            name="bias", trainable=True)

  def call(self, inputs):
    # Define the contraction.
    # We break it out so we can parallelize a batch using
    # tf.vectorized_map (see below).
    def f(input_vec, a_var, b_var, bias_var):
      # Reshape to a matrix instead of a vector.
      input_vec = tf.reshape(input_vec, (32, 32))

      # Now we create the network.
      a = tn.Node(a_var)
      b = tn.Node(b_var)
      x_node = tn.Node(input_vec)
      a[1] ^ x_node[0]
      b[1] ^ x_node[1]
      a[2] ^ b[2]

      # The TN should now look like this
      #   |     |
      #   a --- b
      #    \   /
      #      x

      # Now we begin the contraction.
      c = a @ x_node
      result = (c @ b).tensor

      # To make the code shorter, we also could've used Ncon.
      # The above few lines of code is the same as this:
      # result = tn.ncon([x, a_var, b_var], [[1, 2], [-1, 1, 3], [-2, 2, 3]])

      # Finally, add bias.
      return result + bias_var

    # To deal with a batch of items, we can use the tf.vectorized_map
    # function.
    # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
    result = tf.vectorized_map(
        lambda vec: f(vec, self.a_var, self.b_var, self.bias), inputs)
    return tf.nn.relu(tf.reshape(result, (-1, 1024)))

In [3]:
Dense = tf.keras.layers.Dense
fc_model = tf.keras.Sequential(
    [
     tf.keras.Input(shape=(2,)),
     Dense(1024, activation=tf.nn.relu),
     Dense(1024, activation=tf.nn.relu),
     Dense(1, activation=None)])
fc_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              3072      
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 1,053,697
Trainable params: 1,053,697
Non-trainable params: 0
_________________________________________________________________


In [4]:
tn_model = tf.keras.Sequential(
    [
     tf.keras.Input(shape=(2,)),
     Dense(1024, activation=tf.nn.relu),
     # Here, we replace the dense layer with our MPS.
     TNLayer(),
     Dense(1, activation=None)])
tn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              3072      
_________________________________________________________________
tn_layer (TNLayer)           (None, 1024)              5120      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1025      
Total params: 9,217
Trainable params: 9,217
Non-trainable params: 0
_________________________________________________________________


In [6]:
import numpy as np
def one_edge_at_a_time(a, b):
  node1 = tn.Node(a)
  node2 = tn.Node(b)
  edge1 = node1[0] ^ node2[0]
  edge2 = node1[1] ^ node2[1]
  tn.contract(edge1)
  result = tn.contract(edge2)
  return result.tensor

def use_contract_between(a, b):
  node1 = tn.Node(a)
  node2 = tn.Node(b)
  node1[0] ^ node2[0]
  node1[1] ^ node2[1]
  # This is the same as 
  # tn.contract_between(node1, node2)
  result = node1 @ node2
  return result.tensor

a = np.ones((1000, 1000))
b = np.ones((1000, 1000))
print("Running one_edge_at_a_time")
%timeit one_edge_at_a_time(a, b)
print("Running use_cotract_between")
%timeit use_contract_between(a, b)

Running one_edge_at_a_time
57.7 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running use_cotract_between
3.3 ms ± 451 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
# Here, we will contract the following shaped network.
# O - O
# | X |
# O - O
a = tn.Node(np.ones((2, 2, 2)))
b = tn.Node(np.ones((2, 2, 2)))
c = tn.Node(np.ones((2, 2, 2)))
d = tn.Node(np.ones((2, 2, 2)))
# Make the network fully connected.
a[0] ^ b[0]
a[1] ^ c[1]
a[2] ^ d[2]
b[1] ^ d[1]
b[2] ^ c[2]
c[0] ^ d[0]
# We are using the "greedy" contraction algorithm.
# Other algorithms we support include "optimal" and "branch".

# Finding the optimial contraction order in the general case is NP-Hard,
# so there is no single algorithm that will work for every tensor network.
# However, there are certain kinds of networks that have nice properties that
# we can expliot to making finding a good contraction order easier.
# These types of contraction algorithms are in developement, and we welcome 
# PRs!

# `tn.reachable` will do a BFS to get all of the nodes reachable from a given
# node or set of nodes.
# nodes = {a, b, c, d}
nodes = tn.reachable(a)
result = tn.contractors.greedy(nodes)
print(result.tensor)

tf.Tensor(64.0, shape=(), dtype=float64)


In [8]:
# To make connecting a network a little less verbose, we have included
# the NCon API aswell.

# This example is the same as above.
ones = np.ones((2, 2, 2))
tn.ncon([ones, ones, ones, ones], 
        [[1, 2, 4], 
         [1, 3, 5], 
         [2, 3, 6],
         [4, 5, 6]])

<tf.Tensor: shape=(), dtype=float64, numpy=64.0>

In [9]:
X = np.concatenate([np.random.randn(20, 2) + np.array([3, 3]),
                    np.random.randn(20, 2) + np.array([-3, -3]),
                    np.random.randn(20, 2) + np.array([-3, 3]),
                    np.random.randn(20, 2) + np.array([3, -3])])

Y = np.concatenate([np.ones((40)), -np.ones((40))])

In [10]:
# To specify dangling edges, simply use a negative number on that index.

ones = np.ones((2, 2))
tn.ncon([ones, ones], [[-1, 1], [1, -2]])

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[2., 2.],
       [2., 2.]])>

In [11]:
# To make the singular values very apparent, we will just take the SVD of a
# diagonal matrix.
diagonal_array = np.array([[2.0, 0.0, 0.0],
                           [0.0, 2.5, 0.0],
                           [0.0, 0.0, 1.5]]) 

In [12]:
# First, we will go over the simple split_node method.
a = tn.Node(diagonal_array)
u, vh, _ = tn.split_node(
    a, left_edges=[a[0]], right_edges=[a[1]])
print("U node")
print(u.tensor)
print()
print("V* node")
print(vh.tensor)

U node
tf.Tensor(
[[0.         1.41421356 0.        ]
 [1.58113883 0.         0.        ]
 [0.         0.         1.22474487]], shape=(3, 3), dtype=float64)

V* node
tf.Tensor(
[[0.         1.58113883 0.        ]
 [1.41421356 0.         0.        ]
 [0.         0.         1.22474487]], shape=(3, 3), dtype=float64)


In [13]:
# Now, we can contract u and vh to get back our original tensor!
print("Contraction of U and V*:")
print((u @ vh).tensor)

Contraction of U and V*:
tf.Tensor(
[[2.  0.  0. ]
 [0.  2.5 0. ]
 [0.  0.  1.5]], shape=(3, 3), dtype=float64)


In [14]:
# We can also drop the lowest singular values in 2 ways, 
# 1. By setting max_singular_values. This is the maximum number of the original
# singular values that we want to keep.
a = tn.Node(diagonal_array)
u, vh, truncation_error = tn.split_node(
    a, left_edges=[a[0]], right_edges=[a[1]], max_singular_values=2)
# Notice how the two largest singular values (2.0 and 2.5) remain
# but the smallest singular value (1.5) is removed.
print((u @ vh).tensor)

tf.Tensor(
[[2.  0.  0. ]
 [0.  2.5 0. ]
 [0.  0.  0. ]], shape=(3, 3), dtype=float64)


In [15]:
import numpy as np
import tensornetwork as tn

# Create the nodes
a = tn.Node(np.ones((10,))) 
b = tn.Node(np.ones((10,)))
edge = a[0] ^ b[0] # Equal to tn.connect(a[0], b[0])
final_node = tn.contract(edge)
print(final_node.tensor) # Should print 10.0

tf.Tensor(10.0, shape=(), dtype=float64)
